In [1]:
library(BSgenome.Hsapiens.UCSC.hg38)
library(branchpointer)
library(GenomicRanges)



Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4




In [2]:
g <- BSgenome.Hsapiens.UCSC.hg38::BSgenome.Hsapiens.UCSC.hg38


# branchpoints for PRF38A:


In [14]:
#just to find the coordinates
path='/project/42/pacbio/golong/06-isotools/golong_all/results_2022_12'
regs=read.csv(paste0(path,'/04_differential_splicing/tables/golong_all_2022_12_diff_betabinomial_less_strict_3AS_all.csv'))
regs=regs[regs$chrom %in% paste0('chr',c(1:22,'X','Y')),]
regs[regs$gene=='PRPF38A',]


,gene,gene_id,chrom,strand,start,alternative,end,NMD1,NMD2,X3AS_effect,⋯,M11_all_wt_in_cov,M11_all_wt_total_cov,K01_all_wt_in_cov,K01_all_wt_total_cov,K02_all_wt_in_cov,K02_all_wt_total_cov,N01_all_wt_in_cov,N01_all_wt_total_cov,N02_all_wt_in_cov,N02_all_wt_total_cov
,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>,<fct>,<fct>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
2628,PRPF38A,ENSG00000134748.13,chr1,+,52414647,52414740,52414761,False,False,in_frame,⋯,2,42,5,261,2,159,12,261,5,164
5036,PRPF38A,ENSG00000134748.13,chr1,+,52414647,52414731,52414740,False,False,coding,⋯,0,2,2,5,0,2,2,12,2,5


In [15]:
length(regs$gene)

[1] 7951

In [4]:
ag_pos=c(52414761,52414740,52414731,52414749)
ag_name=c("AG","AG21","AG30","AG12")
queryPRPF38A <- GRanges(seqnames = 'chr1', strand = '+', ranges = IRanges(start = ag_pos-44,width=27),
                       gene_id='PRPF38A', id=ag_name, to_3prime=18, to_5prime=ag_pos-52414647-27, same_gene=FALSE) 
branchpointPredictionsPRPF38A <- predictBranchpoints(queryPRPF38A,queryType = "region",BSgenome = g)

for (which in ag_name){
png(filename=paste0(path,"/07_branchpoint_prediction/plots/branchpointer_images/branchpointer_PRPF38A_",which,"_prim.png"),500,700)
        p=plotBranchpointWindow(which,branchpointPredictionsPRPF38A,probabilityCutoff = 0.52,plotMutated = FALSE,plotStructure = FALSE)
        print(p)
        dev.off()
}
#branchpointPredictionsPRPF38A
#GCTTTGGGGAGGGGGTGGTGGTATTGGTGTTATATGTATATTGCTAACCAGATCTGGTAGTCTGACCAGTCTTTTCTACA

# branchpoints for all 3'AS regions:


In [6]:
path='/project/42/pacbio/golong/06-isotools/golong_all/results_2022_12'
regs=read.csv(paste0(path,'/04_differential_splicing/tables/golong_all_2022_12_diff_betabinomial_less_strict_3AS_all.csv'))
regs=regs[regs$chrom %in% paste0('chr',c(1:22,'X','Y')),]

regs=regs[order(regs$AS_shift),]
regs$bpdist=0
regs$bpdist_alt=0

#select all regions
regs_sel=1:length(regs$AS_shift)#sum(regs$AS_shift<12)+(1:sum(regs$AS_shift<21))
bpscores=matrix(numeric(length(regs_sel)*27),ncol=27, dimnames=list(NULL, paste0('prim_bp',-44:-18)))
u2scores=matrix(numeric(length(regs_sel)*27),ncol=27, dimnames=list(NULL, paste0('prim_U2',-44:-18)))
bpscores_alt=matrix(numeric(length(regs_sel)*27),ncol=27, dimnames=list(NULL, paste0('alt_bp',-44:-18)))
u2scores_alt=matrix(numeric(length(regs_sel)*27),ncol=27, dimnames=list(NULL, paste0('alt_U2',-44:-18)))


In [26]:
#takes 90 minutes, including images
g <- BSgenome.Hsapiens.UCSC.hg38::BSgenome.Hsapiens.UCSC.hg38

for (sel in regs_sel){
    #provide the branchpoint region: -44 to -18
    dist_5prime=regs[sel,'end']-regs[sel,'start']-18
    if (regs[sel,'strand']=='-'){
        bpreg_start=regs[sel,'start']+18
        bpreg_start_alt=regs[sel,'alternative']+18
        dist_5prime_alt=regs[sel,'end']-regs[sel,'alternative']-18        
    }else {
        bpreg_start=regs[sel,'end']-44+1
        bpreg_start_alt=regs[sel,'alternative']-44+1
        dist_5prime_alt=regs[sel,'alternative']-regs[sel,'start']-18
    }
    queryIntron <- GRanges(seqnames = regs[sel,'chrom'], strand = regs[sel,'strand'], ranges = IRanges(start = bpreg_start, width=27),gene_id=regs[sel,'gene_id'], id='prim', to_3prime=18, to_5prime=dist_5prime, same_gene=TRUE) 
    queryIntron_alt <- GRanges(seqnames = regs[sel,'chrom'], strand = regs[sel,'strand'], ranges = IRanges(start = bpreg_start_alt, width=27),gene_id=regs[sel,'gene_id'], id='alt', to_3prime=18, to_5prime=dist_5prime_alt, same_gene=TRUE) 

    branchpointPredictionsIntron <- predictBranchpoints(queryIntron,queryType = "region",BSgenome = g)
    branchpointPredictionsIntron_alt <- predictBranchpoints(queryIntron_alt,queryType = "region",BSgenome = g)

    regs$bpdist[sel]=44-which.max(branchpointPredictionsIntron$branchpoint_prob)
    regs$bpdist_alt[sel]=44-which.max(branchpointPredictionsIntron_alt$branchpoint_prob)
    bpscores[sel,]=branchpointPredictionsIntron$branchpoint_prob
    u2scores[sel,]=branchpointPredictionsIntron$U2_binding_energy
    bpscores_alt[sel,]=branchpointPredictionsIntron_alt$branchpoint_prob
    u2scores_alt[sel,]=branchpointPredictionsIntron_alt$U2_binding_energy
    if (regs[sel,'padj']<.05 || (regs$AS_shift[sel]>=-21 && regs$AS_shift[sel]<=-12)){
        png(filename=paste0(path,"/07_branchpoint_prediction/plots/branchpointer_images/branchpointer_",sprintf("sequence_%04d", regs$AS_shift[sel]),"_",as.character(regs$gene[sel]),"_prim.png"),500,700)
        p=plotBranchpointWindow('prim',branchpointPredictionsIntron,probabilityCutoff = 0.52,plotMutated = FALSE,plotStructure = FALSE,exons = exons)
        print(p)
        dev.off()
        png(filename=paste0(path,"/07_branchpoint_prediction/plots/branchpointer_images/branchpointer_",sprintf("sequence_%04d", regs$AS_shift[sel]),"_",as.character(regs$gene[sel]),"_alt.png"),500,700)
        p=plotBranchpointWindow('alt',branchpointPredictionsIntron_alt,probabilityCutoff = 0.52,plotMutated = FALSE,plotStructure = FALSE,exons = exons)
        print(p)
        dev.off()
    }
}

In [14]:
regs_bp=cbind(regs,bpscores,u2scores,bpscores_alt,u2scores_alt)
write.csv(regs_bp, paste0(path,"/07_branchpoint_prediction/tables/golong_all_2022_12_diff_3AS_all_branchpointer.csv"))


# branchpoints for all introns expressed in K562


In [5]:
# branchpoints for all introns expressed in K562:
introns=read.csv('/project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/01_transcriptome/golong_all_2022_12_K562_introns_10.csv')
introns=introns[introns$chrom %in% paste0('chr',c(1:22,'X','Y')),]
fwd=introns$strand=='+'
introns$wd_start=NA

search_range=c(18,44)

introns$wd_start[fwd]=introns$end[fwd]-search_range[2]+1
introns$wd_start[! fwd]=introns$start[! fwd]+search_range[1]
introns$wd_end=introns$wd_start-search_range[1]+search_range[2]

queryIntron <- GRanges(seqnames = introns$chrom, strand = introns$strand, ranges = IRanges(start = introns$wd_start, introns$wd_end),
                       gene_id=introns$name, id=introns$X, to_3prime=search_range[1], to_5prime=introns$end-introns$start-search_range[1], same_gene=FALSE) 



In [24]:
g <- BSgenome.Hsapiens.UCSC.hg38::BSgenome.Hsapiens.UCSC.hg38
#sel=1:dim(introns)[1]
#introns$bpdist=0
#bpscores=matrix(numeric(dim(introns)[1]*27),ncol=27, dimnames=list(NULL, paste0('prim_bp',-search_range[2]:-search_range[1])))
#u2scores=matrix(numeric(dim(introns)[1]*27),ncol=27, dimnames=list(NULL, paste0('prim_U2',-search_range[2]:-search_range[1])))

sel=sel[sel>i] # comment out all above lines when using this to complete predictions!


for (i in sel){    
    branchpointPredictionsIntron=NULL
    try( #this fails at introns[i,]=105183	GAGE2E	chrX	+	49348386	49528391	49528348	49528374
        branchpointPredictionsIntron <- predictBranchpoints(queryIntron[i,],queryType = "region",BSgenome = g)
    )
    if (is.null(branchpointPredictionsIntron)){
        message('skipping prediction of region ',i)
    }else{
        introns$bpdist[i]=44-which.max(branchpointPredictionsIntron$branchpoint_prob)
        bpscores[i,]=branchpointPredictionsIntron$branchpoint_prob
        u2scores[i,]=branchpointPredictionsIntron$U2_binding_energy
        if (i%%1000==0){
            message('saving intermediate results ',i)
            regs_bp=cbind(introns[1:i,],bpscores[1:i,],u2scores[1:i,])
            write.csv(regs_bp, paste0('/project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/07_branchpoint_prediction/tables/introns_branchpoints_intermediate_th10_',search_range[1],'bp.csv'), row.names=FALSE)
        }
    }
}
regs_bp=cbind(introns,bpscores,u2scores)
write.csv(regs_bp, paste0('/project/42/pacbio/golong/06-isotools/golong_all/results_2022_12/07_branchpoint_prediction/tables/introns_K562_branchpoints_th10_',search_range[1],'bp.csv'), row.names=FALSE)
# about 44000 in total for th100 (at least 100 reads coverage), 
# and 100000 for th10 (10 reads coverage in K562)